# LangGraph Summary-Based Memory Management

This notebook demonstrates **summary-based memory** in LangGraph,
a technique used to maintain conversational context while
preventing unbounded message growth.

Instead of keeping all past messages, the conversation is:
- Summarized into a running abstract
- Old messages are deleted
- The summary is injected back into the prompt

This pattern is commonly used in production conversational agents.


In [129]:
import getpass
import os
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, RemoveMessage, SystemMessage
from langchain_openai import ChatOpenAI
from typing import Literal

In [130]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Defining the Graph State

The graph state extends `MessagesState` by adding a
persistent `summary` field.

- `messages` holds the most recent interaction
- `summary` stores a compressed representation of prior context


In [131]:
class State(MessagesState):
    summary: str

### Verifying Optional Summary Field

The summary field may not exist initially.
A safe `.get()` check ensures graceful handling.


In [132]:
test_state = State()

In [133]:
bool(test_state.get("summary", ""))

## Chat Model Initialization

A lightweight deterministic model is used to
focus on graph behavior rather than response quality.


In [134]:
chat = ChatOpenAI(
    model="gpt-5-nano", 
    temperature=0, 
    model_kwargs= {"text":{"verbosity": 'low'}},
    ) 


## Defining Graph Nodes

Each node:
- Reads from the shared state
- Performs a task
- Returns new state updates


In [135]:
def ask_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_question:")
    
    question = "What is your question?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

In [136]:
def chatbot(state: State) -> State:
    
    print(f"\n-------> ENTERING chatbot:")
    for i in state["messages"]:
        i.pretty_print()
        
    system_message = f'''
    Here's a quick summary of what's been discussed so far:
    {state.get("summary", "")}
    
    Keep this in mind as you answer the next question.
    '''
    
    response = chat.invoke([SystemMessage(system_message)] + state["messages"])
    response.pretty_print()
    
    return State(messages = [response])

After each response, the user decides whether
the conversation should continue.


In [137]:
def ask_another_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_another_question:")
    
    question = "Would you like to ask one more question (yes/no)?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

## Summarizing and Deleting Old Messages

This node:
1. Converts recent messages into text
2. Updates the running summary
3. Deletes all existing messages
4. Preserves context via the summary field


In [138]:
def summarize_and_delete_messages(state: State) -> State:
    print(f"\n-------> ENTERING ENTERING summarize_and_delete_messages:")
    
    new_conversation = ""
    for i in state["messages"]:
        new_conversation += f"{i.type}: {i.content}\n\n"
        
    summary_instructions = f'''
Update the ongoing summary by incorporating the new lines of conversation below.  
Build upon the previous summary rather than repeating it so that the result  
reflects the most recent context and developments.


Previous Summary:
{state.get("summary", "")}

New Conversation:
{new_conversation}
'''
    
    print(summary_instructions)
    
    summary = chat.invoke([HumanMessage(summary_instructions)])
    
    remove_messages = [RemoveMessage(id = i.id) for i in state["messages"][:]]
    
    return State(messages = remove_messages, summary = summary.content)

## Conditional Routing

If the user answers `"yes"`, the conversation is summarized
and restarted. Otherwise, execution ends.


In [139]:
def routing_function(state: State) -> Literal["summarize_and_delete_messages", "__end__"]:
    
    if state["messages"][-1].content == "yes":
        return "summarize_and_delete_messages"
    else:
        return "__end__"

## Building the Graph

The graph combines:
- Conversational flow
- Summary-based memory
- Controlled looping


In [140]:
graph = StateGraph(State)

In [141]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("ask_another_question", ask_another_question)
graph.add_node("summarize_and_delete_messages", summarize_and_delete_messages)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "ask_another_question")
graph.add_conditional_edges(source = "ask_another_question", 
                            path = routing_function)
graph.add_edge("summarize_and_delete_messages", "ask_question")

In [142]:
graph_compiled = graph.compile()

In [143]:
graph_compiled

## Executing the Graph

The graph starts with an empty message history.
Long-term context is preserved through summaries.


In [145]:
graph_compiled.invoke(State(messages = []))

## Summary

This notebook demonstrated:

- Extending `MessagesState` with a summary field
- Summary-based memory for long conversations
- Deleting old messages without losing context
- Injecting summaries into system prompts
- Building production-style conversational agents

Summary-based memory is essential for
scalable and cost-effective LLM systems.
